In [1]:
# Pyart is compatible only with Python 3.8x
import os, pyart, numpy as np, imageio, warnings, matplotlib.pyplot as plt, shutil
warnings.filterwarnings('ignore')
from boto.s3.connection import S3Connection
#%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\vikra\anaconda3\envs\env\lib\site-packages\matplotlib_inline\config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
conn = S3Connection(anon = True)
bucket = conn.get_bucket('noaa-nexrad-level2')

In [3]:
def get_s3_object(bucket, search_prefix, hour, minute, verbose=False):
    '''
        DOCSTRING:
            Creates a list of bucket-items present on S3 with a specified prefix-pattern.
            Loops through these potential candidates to identify the best filename which matches HOUR and MINUTE.
    '''
    if verbose:  print(f'Starting search for: Prefix = "{search_prefix}", Hour = "{HOUR}", Minute = "{MINUTE}"...')
    bucket_list = list(bucket.list(prefix = search_prefix))
    desired_s3_object = None

    for i, s3_fname in enumerate(bucket_list):
        if not s3_fname.name.endswith('gz'):
            continue
        s3_timestamp = s3_fname.name.split('_')[1]
        
        # Found exact match of Hour+Minute for that station
        if s3_timestamp.startswith(''.join([HOUR,MINUTE])):
            if verbose:  print(i, s3_fname.key, bucket_list[i])
            desired_s3_object = s3_fname
            break
        
        # Found partial match of Hour, keep iterating until we find max minute
        if s3_timestamp.startswith(''.join([HOUR, MINUTE[0]])):
            desired_s3_object = s3_fname

    if desired_s3_object is None:
        desired_s3_object = bucket_list[0]

    if verbose:  print(f'Found desired S3 object: {desired_s3_object}...')
    return desired_s3_object

# Year/Month/Day/Station/
#my_pref = '2011/05/20/KVNX/'
FEATURE_TO_VISUALIZE = 'reflectivity'
STATION = 'KVNX'
YEAR = '2011'
MONTH = '05'
DAY = '20'
HOUR = '10'
MINUTE = '59'

my_prefix = '/'.join([YEAR, MONTH, DAY, STATION])+'/'
desired_s3_object = get_s3_object(bucket, my_prefix, HOUR, MINUTE, verbose=True)

Starting search for: Prefix = "2011/05/20/KVNX/", Hour = "10", Minute = "59"...
Found desired S3 object: <Key: noaa-nexrad-level2,2011/05/20/KVNX/KVNX20110520_105809_V06.gz>...


In [4]:
def create_path_if_not_exist(folder, verbose=True):
    """Checks if input path exists, and creates the directory if required.
    
    Args:
        folder ([type]): [description]
        verbose (bool, optional): [description]. Defaults to True.
    """    
    if not os.path.exists(folder):
        if verbose:  print(f'Path doesnt exist. Creating it now... {folder}')
        os.mkdir(folder)

folder_path = os.path.join(os.getcwd(), 'data', STATION)
create_path_if_not_exist(folder_path)

fname = desired_s3_object.name.split("/")[-1]
file_path = os.path.join(folder_path, fname)
print(file_path)

# Write current query file to file_path
desired_s3_object.get_contents_to_filename(filename=file_path)
# Read the queried dataset into radar
radar = pyart.io.read(os.path.join(file_path))

c:\Users\vikra\OneDrive\Desktop\Vikrant\GIT_Repos\scapsulators\Documentation\Temporary-Codebases-for-POCs\Vikrant-Python\Data-Wrangling-Project-1\data\KVNX\KVNX20110520_105809_V06.gz


In [5]:
print(radar.info())

altitude:
	data: <ndarray of type: float64 and shape: (1,)>
	long_name: Altitude
	standard_name: Altitude
	units: meters
	positive: up
altitude_agl: None
antenna_transition: None
azimuth:
	data: <ndarray of type: float64 and shape: (8280,)>
	units: degrees
	standard_name: beam_azimuth_angle
	long_name: azimuth_angle_from_true_north
	axis: radial_azimuth_coordinate
	comment: Azimuth of antenna relative to true north
elevation:
	data: <ndarray of type: float32 and shape: (8280,)>
	units: degrees
	standard_name: beam_elevation_angle
	long_name: elevation_angle_from_horizontal_plane
	axis: radial_elevation_coordinate
	comment: Elevation of antenna relative to the horizontal plane
fields:
	spectrum_width:
		data: <ndarray of type: float32 and shape: (8280, 1832)>
		units: meters_per_second
		standard_name: doppler_spectrum_width
		long_name: Spectrum Width
		valid_max: 63.0
		valid_min: -63.5
		coordinates: elevation azimuth range
		_FillValue: -9999.0
	differential_phase:
		data: <ndarray 

In [5]:
def generate_images_for_animation_by_sweep(radar, station, tgt_folder, sweep_intervals=3, feature_to_plot='reflectivity', image_format='jpeg', verbose=True):
    """Generates a set of images that will be used to generate the GIF file.

    Args:
        radar ([pyart]): pyart-object got after reading the binary S3-file.
        station ([str]): Radar-Station 4-letter code.
        tgt_folder ([str]): Location where the raw images will be written.
        sweep_intervals (int, optional): Interval-size governs number of volume-scans to use for GIF-generation. Defaults to 3.
        feature_to_plot (str, optional): Can be (reflectivity, spectrum_width, velocity).
            <Reflectivity> tells how dense the atmosphere was, and how strong the reflected RADAR signal was in Decibels relative to Z.
            <Doppler-Specturm-Width> tells us how fast moisture particles are moving in atmosphere. Set of particles grouped together form one pixel on Volume-scan.
            <Radial-Velocity-of-scatterers> the "scatterers" are what causes the transmitted signal to be returned to the Radar (eg: aerosols, hydrometeors and refractive index irregularities).
                This measurement corresponds to how fast these "scatterers" are moving away from the Radar.. Defaults to 'reflectivity'.
        image_format (str, optional): Can be (jpeg, png, gif, ...). Defaults to 'jpeg'.
        verbose (bool, optional): Used for controlling logging. Defaults to True.
        Note: PPI is Plan Position Indicator for Radar

    Returns:
        res [list(str)]: List of file-names used for generating the overall GIF.
    """
    dataset_for_sweep = radar.fields[feature_to_plot]['data'].data
    res = []
    my_figure = plt.figure(figsize = [10,8])
    my_display = pyart.graph.RadarMapDisplay(radar)
    
    for sweep_i in range(0,100,sweep_intervals):
        if verbose:  print(f'Producing image for sweep={sweep_i}...')
        try:
            my_display.plot_ppi_map(feature_to_plot, sweep=sweep_i, vmin=dataset_for_sweep.min(), vmax=dataset_for_sweep.max(), raster=True)
            file_name = f'{tgt_folder}/{station}_{sweep_i}.jpeg'
            plt.savefig(file_name)
            res.append(file_name)
        except Exception as e:
            if verbose:  print(f'Breaking from the loop. Exception={e}')
            break
        finally:
            plt.clf()
    if verbose:  print(res)
    return res



TGT_FOLDER = os.path.join('visualizations',STATION)
create_path_if_not_exist(TGT_FOLDER)

RAW_FOLDER = os.path.join(TGT_FOLDER, 'raw')
create_path_if_not_exist(RAW_FOLDER)


images_for_animation = generate_images_for_animation_by_sweep(radar, STATION, RAW_FOLDER, 2, FEATURE_TO_VISUALIZE)
imgs = [imageio.imread(img) for img in images_for_animation]
imageio.mimwrite(os.path.join(TGT_FOLDER,STATION)+'.gif', imgs, fps=2)

shutil.rmtree(RAW_FOLDER)

Path doesnt exist. Creating it now... visualizations\KVNX\raw
Producing image for sweep=0...
Producing image for sweep=2...
Producing image for sweep=4...
Producing image for sweep=6...
Producing image for sweep=8...
Producing image for sweep=10...
Producing image for sweep=12...
Producing image for sweep=14...
Producing image for sweep=16...
Producing image for sweep=18...
Breaking from the loop. Exception=('Sweep out of range: ', 18)
['visualizations\\KVNX\\raw/KVNX_0.jpeg', 'visualizations\\KVNX\\raw/KVNX_2.jpeg', 'visualizations\\KVNX\\raw/KVNX_4.jpeg', 'visualizations\\KVNX\\raw/KVNX_6.jpeg', 'visualizations\\KVNX\\raw/KVNX_8.jpeg', 'visualizations\\KVNX\\raw/KVNX_10.jpeg', 'visualizations\\KVNX\\raw/KVNX_12.jpeg', 'visualizations\\KVNX\\raw/KVNX_14.jpeg', 'visualizations\\KVNX\\raw/KVNX_16.jpeg']


<Figure size 720x576 with 0 Axes>